In [1]:
#This ipynb file utilizes a module called "citipy". The citipy module is first referenced in section
#6.1.5 - Generate Random World Cities. The bootcamp really doesn't describe what it is or what it does specifically.
#The link (https://pypi.org/project/citipy/) provides a means by which to download and install this module, but, like
#this bootcamp, fails to provide any information on what it does or how it works other than
#"Look for nearest city with geo coordinates." There is a GitHub repository for the module at
#https://github.com/wingchen/citipy which seems to have some info. Obviously this bootcamp does not afford adequate time
#to review this resource in order to determine how to use it effectively. 

In [2]:
#*****************************************************************************
#6.1.4 Generate Random Latitudes and Longitudes
#*****************************************************************************
#See section 6.1.4 Generate Random Latitudes and Longitudes under "Create Latitude and Longitude Combinations"
#for the source of this section of code.

# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
#*******
#6.1.4
#*******
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [4]:
#*******
#6.1.4
#*******
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
#*****************************************************************************
#6.1.5 Generate Random World Cities
#*****************************************************************************
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

In [6]:
#*******
#6.1.5
#*******

#Note that in cell 3 of this file 2000 random values were generated for both longitudinal and lattitudinal values.
#These values serve as 2000 sets of coordinates which will be bumped against the citipy module in order to identify
#which cities are closest to the randomly generated coordinates. The code below appears to filter out any cities that
#are selected more than once. For example, if the closest city to two pairs of coordinates is Des Moines, then
#only one instance of "Des Moines" is added to the list. This appararently happens a lot as there are only roughly 750
#cities identified from a set of 2000 coordinates. Makes you wonder how comprehensive this module is. After all, what
#consitutes a "city", right?

#Anyway, the output of this cell is a list of cities

# Create a list for holding the cities.
cities = []

# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count.
len(cities)

767

In [7]:
#This next segment of code will bump the list of cities against a resource at api.openweathermap.org.
#For this API to work, an account had to be created at https://home.openweathermap.org/ in order to acquire
#an API key. That key value is stored in the config.py file found in the same directory as this ipynb file.
#See section 6.2.3 Make an API Call of the bootcamp for the source of the code below.

In [8]:
#*****************************************************************************
#6.2.3 Make an API Call
#*****************************************************************************
# Import the datetime module from the datetime library.
from datetime import datetime

# Import the requests library.
import requests

# Import the API key from the config file.
from config import weather_api_key

#This variable serves as the baseline url used in retrieving data from the source. 
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [9]:
#*******
#6.2.3
#*******
#This segment of code loops through all the cities and builds a unique url for each one using the
#url variable in the previous cell. Honestly, I don't know what else it does. Are those urls stored in
#a list or something? Couldn't tell you. We just copy and paste code in this bootcamp.

for i, city in enumerate(cities):
    city_url = url + "&q=" + city.replace(" ","+")
    #print(city_url)

In [10]:
#This next segment of code comes from 6.2.6 Get the City Weather Data. This segement of code seems to build
#on the cells above. See the description above for 6.2.3.

In [11]:
#*****************************************************************************
#6.2.6 Get the City Weather Data
#*****************************************************************************
# Create an empty list to hold the weather data.
city_data = []

# Create counters.
record_count = 1
set_count = 1

In [12]:
#*****
#6.2.6
#*****
#This code loops through all the cities in the list and bumps each one against the openweathermap source
#by producing a unique url for the city using the url prefix established above. The response from
#openweathermap is data JSON format. Each city has its own JSON formatted data. The code below extracts select
#pieces of the returned data (ex, latitude of the city, max temperature, wind speed, etc) and stores that data
#in a list.

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()

        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]

#Why is this "[0]" here? I have no idea but if it ain't this whole thing doesn't work. Sure would be nice to know,
#but... again... we just cut and paste code in this bootcamp. Good thing I'm really smart and patient enough to 
#spend 3 hours on my Memorial Day weekend deconstructing the stupid (and I do mean STUPID) and counter-productive
#error handling developed for this exercise. Anyway, be sure to look into what "[0]" means when you have a chance.
        
        # Convert the date to ISO standard.
        #city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')

        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description})

#This stupid error handling treats every "error" code as though it's a 404 error. Honestly, how dumb is that?
#What about a typo in the code that produces some other kind of error? Plausible, considering this exercise is
#for people learning Python. Anyone stop to think "Hey, maybe we should actually write error handling routine
#that only "skips the city" if there is a 400 error and returns all other errors, in case, you know, a student
#makes a mistake in the code and wants to actually Google the freakin' error code like a real progammer? Guess
#that's too much to ask. "Nah, leave that to the neophyte to do.""

#Anyway, so what is happening here is the code is ignoring any "cities" that don't prompt a return JSON dataset
#from openweathermap. So the list of 750 or so cities gets pared down to probably 700 or so. Honestly, I don't
#know how that's professional when you're attempting to generate "random" results but continually dump data that
#doesn't work. But hey, whatever.

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Processing Record 1 of Set 1 | punta arenas
Processing Record 2 of Set 1 | pemangkat
Processing Record 3 of Set 1 | busselton
Processing Record 4 of Set 1 | pangai
Processing Record 5 of Set 1 | rikitea
Processing Record 6 of Set 1 | ushuaia
Processing Record 7 of Set 1 | kangaatsiaq
Processing Record 8 of Set 1 | carnarvon
Processing Record 9 of Set 1 | san marcos de colon
Processing Record 10 of Set 1 | masterton
Processing Record 11 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 12 of Set 1 | marcona
City not found. Skipping...
Processing Record 13 of Set 1 | lagoa
Processing Record 14 of Set 1 | pisco
Processing Record 15 of Set 1 | albany
Processing Record 16 of Set 1 | pemba
Processing Record 17 of Set 1 | kudahuvadhoo
Processing Record 18 of Set 1 | mahajanga
Processing Record 19 of Set 1 | oriximina
Processing Record 20 of Set 1 | collie
Processing Record 21 of Set 1 | hilo
Processing Record 22 of Set 1 | tiznit
Processing Record 23 of Set 1 | zharkovskiy


Processing Record 38 of Set 4 | honningsvag
Processing Record 39 of Set 4 | makakilo city
Processing Record 40 of Set 4 | nizwa
Processing Record 41 of Set 4 | cetraro
Processing Record 42 of Set 4 | moranbah
Processing Record 43 of Set 4 | lucea
Processing Record 44 of Set 4 | kavieng
Processing Record 45 of Set 4 | caravelas
Processing Record 46 of Set 4 | barrow
Processing Record 47 of Set 4 | hithadhoo
Processing Record 48 of Set 4 | viligili
City not found. Skipping...
Processing Record 49 of Set 4 | svetlaya
Processing Record 50 of Set 4 | camopi
Processing Record 1 of Set 5 | kropotkin
Processing Record 2 of Set 5 | sambava
Processing Record 3 of Set 5 | birao
Processing Record 4 of Set 5 | salalah
Processing Record 5 of Set 5 | lerwick
Processing Record 6 of Set 5 | najran
Processing Record 7 of Set 5 | yellowknife
Processing Record 8 of Set 5 | baruun-urt
Processing Record 9 of Set 5 | victoria
Processing Record 10 of Set 5 | saldanha
Processing Record 11 of Set 5 | sisimiut
P

Processing Record 28 of Set 8 | jahrom
City not found. Skipping...
Processing Record 29 of Set 8 | mar del plata
Processing Record 30 of Set 8 | opuwo
Processing Record 31 of Set 8 | port hawkesbury
Processing Record 32 of Set 8 | carauari
Processing Record 33 of Set 8 | nacozari
Processing Record 34 of Set 8 | yokadouma
Processing Record 35 of Set 8 | tunduru
City not found. Skipping...
Processing Record 36 of Set 8 | pacific grove
Processing Record 37 of Set 8 | broken hill
Processing Record 38 of Set 8 | narsaq
Processing Record 39 of Set 8 | praia da vitoria
Processing Record 40 of Set 8 | nagtipunan
Processing Record 41 of Set 8 | barabai
Processing Record 42 of Set 8 | khani
Processing Record 43 of Set 8 | caldwell
Processing Record 44 of Set 8 | westport
Processing Record 45 of Set 8 | ayagoz
Processing Record 46 of Set 8 | esperance
Processing Record 47 of Set 8 | mikuni
Processing Record 48 of Set 8 | ancud
Processing Record 49 of Set 8 | leningradskiy
Processing Record 50 of 

Processing Record 15 of Set 12 | ospitaletto
Processing Record 16 of Set 12 | mugla
Processing Record 17 of Set 12 | lagodekhi
Processing Record 18 of Set 12 | port moresby
Processing Record 19 of Set 12 | suzhou
Processing Record 20 of Set 12 | lazaro cardenas
Processing Record 21 of Set 12 | laurel
Processing Record 22 of Set 12 | sao filipe
Processing Record 23 of Set 12 | timizart
Processing Record 24 of Set 12 | porto novo
Processing Record 25 of Set 12 | longhua
Processing Record 26 of Set 12 | sterling
Processing Record 27 of Set 12 | udachnyy
Processing Record 28 of Set 12 | bergisch gladbach
Processing Record 29 of Set 12 | les cayes
Processing Record 30 of Set 12 | caiaponia
Processing Record 31 of Set 12 | ngunguru
Processing Record 32 of Set 12 | nisia floresta
Processing Record 33 of Set 12 | neijiang
Processing Record 34 of Set 12 | myatlevo
Processing Record 35 of Set 12 | sept-iles
Processing Record 36 of Set 12 | horqueta
Processing Record 37 of Set 12 | galiwinku
City

Processing Record 2 of Set 16 | las palmas
Processing Record 3 of Set 16 | san jeronimo
Processing Record 4 of Set 16 | prainha
Processing Record 5 of Set 16 | tshane
Processing Record 6 of Set 16 | impfondo
Processing Record 7 of Set 16 | lisakovsk
Processing Record 8 of Set 16 | barawe
City not found. Skipping...
Processing Record 9 of Set 16 | bafra
Processing Record 10 of Set 16 | payakumbuh
Processing Record 11 of Set 16 | mpongwe
Processing Record 12 of Set 16 | elko
Processing Record 13 of Set 16 | lac du bonnet
Processing Record 14 of Set 16 | college
Processing Record 15 of Set 16 | locri
Processing Record 16 of Set 16 | santa cruz
Processing Record 17 of Set 16 | spirit river
-----------------------------
Data Retrieval Complete      
-----------------------------


In [13]:
#*****************************************************************************
#6.2.7 Create a DataFrame of City Weather Data
#*****************************************************************************

# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)

#Re-order the columns
new_column_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_column_order]

# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"

# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="")